In [84]:
from sklearn.metrics import ndcg_score, dcg_score
import numpy as np
from scipy.linalg import svd

In [1]:
true = np.asarray([[2, 4, 1, 1, 1]])
relevance = np.asarray([[2, 5, 2, 3, 1]])
print(ndcg_score(true, relevance))

0.9748317848747132


In [38]:
import pandas as pd
import math

In [3]:
ratings = pd.read_csv('data/dst-3.0_mathml_14_5_rating.csv')
movies = pd.read_csv('data/movie.csv')

In [6]:
df = pd.merge(ratings, movies, how='left', on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [7]:
df['title'].value_counts(ascending=False).nlargest(5)

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: title, dtype: int64

In [12]:
df.groupby('title', as_index=False)['rating'].mean().sort_values(by=['rating', 'title'], ascending=[False, False])

,title,rating
26482,Yonkers Joe (2008),5.0
26430,Year Zero: The Silent Death of Cambodia (1979),5.0
25918,Who Killed Vincent Chin? (1987),5.0
25751,When I Walk (2013),5.0
25607,Welcome to Australia (1999),5.0
...,...,...
327,6 Desires: DH Lawrence and Sardinia (2014),0.5
225,3 Little Ninjas and the Lost Treasure (1990),0.5
223,3 Holiday Tails (Golden Christmas 2: The Secon...,0.5
155,20 Years After (2008),0.5


In [19]:
mask = df.groupby('title', as_index=False)['rating'].agg(['mean', 'count']).sort_values(by=['count', 'mean'], ascending=[False, False])
mask[mask['count'] > 50]['count'].count()

10472

In [22]:
mask[mask['count'] > 50].sort_values(by=['mean', 'count'], ascending=[False, False])

,mean,count
title,,
"Shawshank Redemption, The (1994)",4.446990,63366
"Godfather, The (1972)",4.364732,41355
"Usual Suspects, The (1995)",4.334372,47006
Schindler's List (1993),4.310175,50054
"Godfather: Part II, The (1974)",4.275641,27398
...,...,...
Barney's Great Adventure (1998),1.163484,419
Glitter (2001),1.124088,685
Bratz: The Movie (2007),1.105556,180


ПРАКТИКА

In [23]:
interactions_df = pd.read_csv('data/users_interactions.csv')
interactions_df.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [24]:
articles_df = pd.read_csv('data/shared_articles.csv')
articles_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [25]:
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.shape

(3047, 13)

In [26]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [27]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [28]:
interactions_df['eventType_weight'] = interactions_df['eventType'].map(event_type)
interactions_df['eventType_weight'].unique()

array([1. , 3. , 2.5, 2. , 4. ])

In [29]:
interactions_df['eventType_weight'].mean()

1.2362885828078327

In [34]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

print(len(users_with_enough_interactions_df))

1140


In [37]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]
print(interactions_from_selected_users_df.shape)

(69868, 9)


In [39]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [40]:
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventType_weight.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df['last_timestamp'].mean()

1470605340.0403006

In [45]:
train_df = interactions_full_df.loc[interactions_full_df['last_timestamp'] < 1475519545]
test_df = interactions_full_df.loc[interactions_full_df['last_timestamp'] >= 1475519545]
train_df.shape

(29325, 4)

In [46]:
train_df

,personId,contentId,eventType_weight,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054096
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


In [47]:
final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


In [48]:
popular = (
    train_df
    .groupby('contentId')
    .eventType_weight.sum().reset_index()
    .sort_values('eventType_weight', ascending=False)
    .contentId.values
)
popular[0]

'-6783772548752091658'

In [52]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)
def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()

In [54]:
np.round(calc_precision('popular'), 3)

0.006

In [57]:
ratings = pd.pivot_table(
    train_df,
    columns='contentId',
    index='personId',
    values='eventType_weight'
    ).fillna(0)
ratings

contentId,-1006791494035379303,-1021685224930603833,-1022885988494278200,-1024046541613287684,-1033806831489252007,-1038011342017850,-1039912738963181810,-1046621686880462790,-1051830303851697653,-1055630159212837930,...,9217155070834564627,921770761777842242,9220445660318725468,9222265156747237864,943818026930898372,957332268361319692,966067567430037498,972258375127367383,980458131533897249,98528655405030624
personId,,,,,,,,,,,,,,,,,,,,,
-1007001694607905623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
-1032019229384696495,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,2.321928,0.0,0.0,0.0,0.0,0.0
-108842214936804958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,2.0,0.0,0.0,0.0
-1130272294246983140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
-1160159014793528221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953707509720613429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
983095443598229476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
989049974880576288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [58]:
round(ratings.loc["-1032019229384696495", "943818026930898372"], 2)

2.32

In [60]:
ratings_m = ratings.values
ratings_m.mean().round(3)

0.017

In [62]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

for i in (range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
     
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]


C:\Users\vadim.kopytko\AppData\Roaming\Python\Python310\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\vadim.kopytko\AppData\Roaming\Python\Python310\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\vadim.kopytko\AppData\Roaming\Python\Python310\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\vadim.kopytko\AppData\Roaming\Python\Python310\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\vadim.kopytko\AppData\Roaming\Python\Python310\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\vadim.kopytko\AppData\Roaming\Python\Python310\site-packages\numpy\lib\functi

In [63]:
similarity_users[3, 40].round(2)

-0.33

In [79]:
prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, final_df.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))
final_df['prediction_user_based'] = prediction_user_based
prediction_user_based[35][0]

'-5148591903395022444'

In [81]:
np.round(calc_precision('prediction_user_based'), 3)

0.005

In [85]:
U, sigma, V = svd(ratings)
U.max()

0.7071067811865497

In [86]:
k = 100
s = np.diag(sigma[:k])
U = U[:, 0:k]
V = V[0:k, :]

round(s.sum(), 2)

2096.43

In [87]:
new_ratings = pd.DataFrame(
    U.dot(s).dot(V), index=ratings.index, columns=ratings.columns
)
top_k = 10
predictions = []

for personId in final_df.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                ~np.in1d(prediction, final_df.loc[personId, "true_train"])
            ]
        )[:top_k]
    )

final_df["prediction_svd"] = predictions

calc_precision("prediction_svd")

0.012023152289763838

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

In [89]:
ratings_matrix = csr_matrix(ratings)

In [90]:
train_df, test_df = train_test_split(ratings_matrix, test_size=0.3, random_state=13)

In [ ]:
model = LightFM(loss='warp', #определяем функцию потерь
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели

In [ ]:
model.fit(train)
prec_score = precision_at_k(model, test).mean()
print(round(prec_score, 2))